In [5]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [82]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data
import xarray as xr
import h5py

# Helpful
import time
import datetime
import itertools
from itertools import product

# My Methods
import importlib
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp

<IPython.core.display.Javascript object>

### Goal of this notebook: Train EMOS global
1. Train one EMOS model globally on the whole grid for each lead time (1...31) and variable(0...4) total 155 Models

#### 0. Basics

In [16]:
var_names = ["u10", "v10", "t2m", "t850", "z500"]

<IPython.core.display.Javascript object>

In [85]:
# Variables to make testing easier --> determines variable and lead_time
var = 0 # 0,...,4
lead_time = 0 # 0,...,30

<IPython.core.display.Javascript object>

#### 1. Load Dataset

In [13]:
dat_train_proc = ldp.load_data_all_train_proc_norm()
dat_test_proc = ldp.load_data_all_test_proc_norm()

<IPython.core.display.Javascript object>

#### 2. Data Split

In [83]:
X_train_var_lead_all, y_train_var_lead_all = split_var_lead(dat_train_proc)
X_test_lead_all, y_test_var_lead_all = split_var_lead(dat_test_proc)

<IPython.core.display.Javascript object>

#### 3. Baseline

In [92]:
crps_baseline = crps_normal(
    mu=X_test_lead_all[var][lead_time].isel(mean_std=0).values,
    sigma=X_test_lead_all[var][lead_time].isel(mean_std=1).values,
    y=y_test_var_lead_all[var][lead_time].values,
) # Can use .values or simply the dataArrray

<IPython.core.display.Javascript object>

In [94]:
crps_baseline.mean()

0.011513211987230255

<IPython.core.display.Javascript object>

#### 4. Train global Emos

In [103]:
EMOS_glob = build_EMOS_network_keras(compile=True)

<IPython.core.display.Javascript object>

In [106]:
EMOS_glob.fit(
    [
        X_train_var_lead_all[var][lead_time].isel(mean_std=0).values.flatten(),
        X_train_var_lead_all[var][lead_time].isel(mean_std=1).values.flatten(),
    ],
    y_train_var_lead_all[var][lead_time].values.flatten(),
    batch_size=5000,
    epochs=5,
    validation_split=0.2,
)

Epoch 1/5
3567/3567 [==============================] - 11s 3ms/step - loss: 0.0191 - val_loss: 0.0131
Epoch 2/5
3567/3567 [==============================] - 11s 3ms/step - loss: 0.0191 - val_loss: 0.0132
Epoch 3/5
3567/3567 [==============================] - 11s 3ms/step - loss: 0.0191 - val_loss: 0.0109
Epoch 4/5
3567/3567 [==============================] - 11s 3ms/step - loss: 0.0191 - val_loss: 0.0195
Epoch 5/5
3567/3567 [==============================] - 11s 3ms/step - loss: 0.0190 - val_loss: 0.0108


<IPython.core.display.Javascript object>

In [ ]:
preds = EMOS_glob.predict(
    [
        X_test_lead_all[var][lead_time].isel(mean_std=0).values.flatten(),
        X_test_lead_all[var][lead_time].isel(mean_std=1).values.flatten(),
    ], verbose = 1
)

148581/174038 [========================>.....] - ETA: 32s

In [ ]:
crps_emos = crps_normal(
    mu=preds[:, 0], sigma=preds[:, 1], y=y_test_var_lead_all[var][lead_time].values.flatten()
)

In [ ]:
crps_emos.mean()